In [1]:
from sift1m_dataset_builder import Builder
import tensorflow_datasets.public_api as tfds
from etils.epath import Path
import numpy as np

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import faiss
import torch

c:\Users\John\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
path = Path("B:/random ML datasets/dataset")
download_manager = tfds.download.DownloadManager(download_dir=path)

In [21]:
b = Builder()
split = b._split_generators(download_manager)

embeddingsGen = split["database"]

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 153.53 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 105.10 url/s]


In [22]:
tensor_list = []
i = 0

for tup in embeddingsGen:
    dict = tup[1]
    tensor_list.append(torch.tensor(dict["embedding"]))
    i=i+1
    if i % 50000 == 0:
        print(i)

combined_tensor = torch.stack(tensor_list)

pq_input_array = combined_tensor.numpy()

50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000


In [23]:
print(pq_input_array)
print(pq_input_array.shape[1])



[[  0.  16.  35. ...  25.  23.   1.]
 [ 14.  35.  19. ...  11.  21.  33.]
 [  0.   1.   5. ...   4.  23.  10.]
 ...
 [ 30.  12.  12. ...  50.  10.   0.]
 [  0.   5.  12. ...   1.   2.  13.]
 [114.  31.   0. ...  25.  16.   0.]]
128


In [14]:
dim = pq_input_array.shape[1]
numSubVectors = 8
subVectorBits = 8

pq = faiss.IndexPQ(dim, numSubVectors, subVectorBits)

In [15]:
pq.train(pq_input_array)

In [16]:
pq.add(pq_input_array)

In [17]:
k = 10  
distances, index = pq.search(pq_input_array[:10], k)

In [18]:
print(distances)
print(index)

[[19890.656 35641.41  35940.766 37764.992 39859.07  40960.75  42346.03
  45371.03  45506.082 47077.29 ]
 [17152.855 28981.86  30410.84  32897.1   40955.78  41304.99  41353.797
  41561.242 41817.492 41827.53 ]
 [16306.107 26835.984 31727.46  32955.586 33282.043 33584.977 33938.65
  34051.414 34066.57  35424.727]
 [18342.58  26773.863 31244.035 34709.293 36348.89  36804.71  36837.83
  37230.62  37981.734 38678.227]
 [15489.635 33504.04  34986.164 35055.14  36468.562 36593.742 36680.844
  37729.03  38740.016 38914.195]
 [17158.016 29218.64  35154.46  35902.484 35902.484 36024.523 36241.883
  37539.977 37801.984 38110.78 ]
 [27275.99  35146.445 43460.652 46560.188 48121.797 48271.984 48414.688
  48819.477 49835.71  50383.742]
 [25039.281 40667.6   42268.64  42574.637 45034.03  45094.83  45142.79
  45563.047 46172.438 46572.652]
 [17948.871 19449.863 22320.523 33307.918 34146.535 35640.23  36739.133
  40383.03  40779.832 42377.39 ]
 [25596.781 27175.95  31074.238 38139.14  38223.31  38462.3